In [1]:
# Import Dependencies
import pandas as pd

In [2]:
# Read the CSV into a Pandas DataFrame
ufo_df = pd.read_csv('Resources/ufoSightings.csv', low_memory=False)

# Remove the rows with missing data
clean_ufo_df = ufo_df.dropna(how="any")

# Converting the "duration (seconds)" column's values to numeric
converted_ufo_df = clean_ufo_df.copy()
converted_ufo_df["duration (seconds)"] = converted_ufo_df.loc[:, "duration (seconds)"].astype(float)

# Display the DataFrame
converted_ufo_df.head()

,datetime,city,state,country,shape,duration (seconds),duration (hours/min),comments,date posted,latitude,longitude
0,10/10/1949 20:30,san marcos,tx,us,cylinder,2700.0,45 minutes,This event took place in early fall around 194...,4/27/2004,29.8830556,-97.941111
3,10/10/1956 21:00,edna,tx,us,circle,20.0,1/2 hour,My older brother and twin sister were leaving ...,1/17/2004,28.9783333,-96.645833
4,10/10/1960 20:00,kaneohe,hi,us,light,900.0,15 minutes,AS a Marine 1st Lt. flying an FJ4B fighter/att...,1/22/2004,21.4180556,-157.803611
5,10/10/1961 19:00,bristol,tn,us,sphere,300.0,5 minutes,My father is now 89 my brother 52 the girl wit...,4/27/2007,36.595,-82.188889
7,10/10/1965 23:45,norwalk,ct,us,disk,1200.0,20 minutes,A bright orange color changing to reddish colo...,10/2/1999,41.1175,-73.408333


In [3]:
# Add double brackets around the column to for the aggregation to create a DataFrame.
grouped_ufo_duration_shape = converted_ufo_df.groupby("shape")[["duration (seconds)"]].mean()
grouped_ufo_duration_shape.head(10)

,duration (seconds)
shape,
changed,3600.000000
changing,2111.616031
chevron,472.417782
cigar,2148.050379
circle,3650.816269
cone,1643.704280
crescent,37800.000000
cross,752.025381
cylinder,3954.055607


In [4]:
# The agg() function can be used for aggregation. 
grouped_ufo_duration_shape = converted_ufo_df.groupby("shape").agg({"duration (seconds)":'mean'})
grouped_ufo_duration_shape.head(10)

,duration (seconds)
shape,
changed,3600.000000
changing,2111.616031
chevron,472.417782
cigar,2148.050379
circle,3650.816269
cone,1643.704280
crescent,37800.000000
cross,752.025381
cylinder,3954.055607


### Multiple Aggregations

In [5]:
# The agg() function can be used to pass more than one aggregation.
ufo_shape_avg_sum = converted_ufo_df.groupby("shape")[["duration (seconds)"]].agg(['mean','sum'])
ufo_shape_avg_sum.head(10)

duration (seconds)             
                       mean          sum
shape                                   
changed         3600.000000      3600.00
changing        2111.616031   3490501.30
chevron          472.417782    402499.95
cigar           2148.050379   3688202.50
circle          3650.816269  23383478.20
cone            1643.704280    422432.00
crescent       37800.000000     37800.00
cross            752.025381    148149.00
cylinder        3954.055607   4266426.00
delta           2307.857143     16155.00

### Grouping on Multiple Columns with Multiple Aggregations

In [6]:
# It is also possible to group a DataFrame by multiple columns
# This returns an object with multiple indices, however, which can be harder to deal with.
# Get the average duration in seconds of UFOs by Country and State.
country_state_avg_duration = converted_ufo_df.groupby(['country', 'state'])[["duration (seconds)"]].mean()

# Display the DataFrame.
country_state_avg_duration.head(10)

duration (seconds)
country state                    
au      al             900.000000
        dc             300.000000
        nt             180.000000
        oh             180.000000
        sa             152.500000
        wa             225.000000
        yt              30.000000
ca      ab            1869.697183
        bc             948.236071
        mb            1291.387097

In [7]:
# The agg() function can be used to pass more than one aggregation.
country_state_duration_metrics = converted_ufo_df.groupby(['country', 'state'])[["duration (seconds)"]].agg(['count','mean','sum'])
country_state_duration_metrics.head(10)

duration (seconds)                        
                           count         mean        sum
country state                                           
au      al                     1   900.000000     900.00
        dc                     1   300.000000     300.00
        nt                     2   180.000000     360.00
        oh                     1   180.000000     180.00
        sa                     2   152.500000     305.00
        wa                     2   225.000000     450.00
        yt                     1    30.000000      30.00
ca      ab                   284  1869.697183  530994.00
        bc                   677   948.236071  641955.82
        mb                   124  1291.387097  160132.00

### Flattening Multi-Indexed Columns to Single Columns

In [8]:
# One method of flattening the MultiIndex columns to a single column: use the to_flat_index() on the columns.
country_state_duration_flatten = country_state_duration_metrics.copy()

# Get the columns after apply the to_flat_index()
country_state_duration_flatten.columns = country_state_duration_flatten.columns.to_flat_index()

# Display the columns
country_state_duration_flatten.columns

Index([('duration (seconds)', 'count'),  ('duration (seconds)', 'mean'),
         ('duration (seconds)', 'sum')],
      dtype='object')

In [9]:
# Use a list comprehension to join the each tuple for each column. 
country_state_duration_flatten.columns = ['_'.join(column) for column in country_state_duration_metrics.columns]
# Display the flattened DataFrame
country_state_duration_flatten

duration (seconds)_count  duration (seconds)_mean  \
country state                                                      
au      al                            1               900.000000   
        dc                            1               300.000000   
        nt                            2               180.000000   
        oh                            1               180.000000   
        sa                            2               152.500000   
...                                 ...                      ...   
us      vt                          254              1042.462598   
        wa                         3707             15273.474357   
        wi                         1205              1928.422656   
        wv                          438              6791.901826   
        wy                          169              1487.828402   

               duration (seconds)_sum  
country state                          
au      al                     900.00  
        dc                     300.00  
        nt                     360.00  
        oh                     180.00  
        sa                     305.00  
...                               ...  
us      vt                  264785.50  
        wa                56618769.44  
        wi                 2323749.30  
        wv                 2974853.00  
        wy                  251443.00  

[84 rows x 3 columns]

In [10]:
# The second method for flattening the multiIndex to one column.
# Get the first level of the multi-index
level_0 = country_state_duration_metrics.columns.get_level_values(0)
print(level_0)
# Get the second level of the multi-index
level_1 = country_state_duration_metrics.columns.get_level_values(1)
print(level_1)

Index(['duration (seconds)', 'duration (seconds)', 'duration (seconds)'], dtype='object')
Index(['count', 'mean', 'sum'], dtype='object')


In [11]:
# Combine the levels and display the DataFrame. 
country_state_duration_metrics.columns = level_0 +"_"+ level_1
country_state_duration_metrics

duration (seconds)_count  duration (seconds)_mean  \
country state                                                      
au      al                            1               900.000000   
        dc                            1               300.000000   
        nt                            2               180.000000   
        oh                            1               180.000000   
        sa                            2               152.500000   
...                                 ...                      ...   
us      vt                          254              1042.462598   
        wa                         3707             15273.474357   
        wi                         1205              1928.422656   
        wv                          438              6791.901826   
        wy                          169              1487.828402   

               duration (seconds)_sum  
country state                          
au      al                     900.00  
        dc                     300.00  
        nt                     360.00  
        oh                     180.00  
        sa                     305.00  
...                               ...  
us      vt                  264785.50  
        wa                56618769.44  
        wi                 2323749.30  
        wv                 2974853.00  
        wy                  251443.00  

[84 rows x 3 columns]

In [12]:
# Get the new column names to rename the columns
country_state_duration_metrics.columns

Index(['duration (seconds)_count', 'duration (seconds)_mean',
       'duration (seconds)_sum'],
      dtype='object')

In [13]:
# Rename the columns in a multi-index DataFrame
country_state_duration_metrics = country_state_duration_metrics.rename(columns={"duration (seconds)_count": "Number of Sightings",
                                                                     "duration (seconds)_mean": "Avg Duration(seconds)",
                                                                     "duration (seconds)_sum": "Total Duration(seconds)"}, level=0)
country_state_duration_metrics.head(10)

Number of Sightings  Avg Duration(seconds)  \
country state                                               
au      al                       1             900.000000   
        dc                       1             300.000000   
        nt                       2             180.000000   
        oh                       1             180.000000   
        sa                       2             152.500000   
        wa                       2             225.000000   
        yt                       1              30.000000   
ca      ab                     284            1869.697183   
        bc                     677             948.236071   
        mb                     124            1291.387097   

               Total Duration(seconds)  
country state                           
au      al                      900.00  
        dc                      300.00  
        nt                      360.00  
        oh                      180.00  
        sa                      305.00  
        wa                      450.00  
        yt                       30.00  
ca      ab                   530994.00  
        bc                   641955.82  
        mb                   160132.00